The main of this notebook is to enter drivendata ds context; link below:
https://www.drivendata.org/competitions/57/nepal-earthquake/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, LabelEncoder,MinMaxScaler, KBinsDiscretizer \
                        , OrdinalEncoder, PolynomialFeatures, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV \
                        , StratifiedKFold, train_test_split
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier \
                        , RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from sklearn.pipeline import make_pipeline, Pipeline
import category_encoders as ce

In [23]:
X = pd.read_csv('train_values.csv')
y = pd.read_csv('train_labels.csv')

In [24]:
from sklearn.utils import class_weight

In [25]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y['damage_grade']),y=y['damage_grade'])

In [26]:
class_weights

array([3.45753065, 0.58591384, 0.9959756 ])

In [27]:
df = pd.concat((X, y['damage_grade']),axis=1)

In [28]:
dummy_regions = pd.get_dummies(df['geo_level_1_id'], prefix='region1', )

In [29]:
df.shape, dummy_regions.shape

((260601, 40), (260601, 31))

In [9]:
df = pd.concat((df, dummy_regions), axis=1)

In [10]:
df.shape

(260601, 71)

In [11]:
df.drop(['building_id', 'geo_level_1_id'], axis=1, inplace=True)

In [32]:
X= df.drop('damage_grade', axis=1)
y= df['damage_grade']

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.2, train_size=0.8, random_state=1, shuffle=True)

In [34]:
est = KBinsDiscretizer(n_bins=40, encode='ordinal', strategy= 'uniform')
est.fit(X_train[['age']])

KBinsDiscretizer(encode='ordinal', n_bins=40, strategy='uniform')

In [35]:
X_train['age'] = est.transform(X_train[['age']])

E:\Tolu\PROGRAMS\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
X_val['age'] = est.fit_transform(X_val[['age']])


E:\Tolu\PROGRAMS\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((208480, 39), (52121, 39), (208480,), (52121,))

In [38]:
X_train[['geo_level_2_id','geo_level_3_id']] = X_train[['geo_level_2_id','geo_level_3_id']].astype('object')

E:\Tolu\PROGRAMS\Anaconda2\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [39]:
X_val[['geo_level_2_id','geo_level_3_id']] = X_val[['geo_level_2_id','geo_level_3_id']].astype('object')

# Target Encode geo_level_1

In [40]:
X_train[['geo_level_1_id','geo_level_2_id','geo_level_3_id']] = X_train[['geo_level_1_id','geo_level_2_id','geo_level_3_id']].astype('object')

In [41]:
X_val[['geo_level_1_id','geo_level_2_id','geo_level_3_id']] = X_val[['geo_level_1_id','geo_level_2_id','geo_level_3_id']].astype('object')

In [42]:
cate_cols = X_train.select_dtypes(exclude='number').columns.tolist()
cate_cols

['geo_level_1_id',
 'geo_level_2_id',
 'geo_level_3_id',
 'land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

In [43]:
val_cate_cols = X_val.select_dtypes(exclude='number').columns.tolist()
val_cate_cols

['geo_level_1_id',
 'geo_level_2_id',
 'geo_level_3_id',
 'land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

In [44]:
te = ce.TargetEncoder()

In [45]:
X_train[cate_cols] = te.fit_transform(X_train[cate_cols], y_train)

In [46]:
X_val[val_cate_cols] = te.transform(X_val[val_cate_cols])

In [76]:
X_val.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
20759,1.938499,2.155779,2.000000,2,0.0,5,4,2.234089,2.329684,2.270366,...,0,0,0,0,0,0,0,0,0,0
136741,1.730462,1.737226,1.800000,2,0.0,9,6,2.234089,2.329684,2.270366,...,0,0,0,0,0,0,0,0,0,0
145104,2.339942,2.353499,2.354839,2,1.0,7,6,2.234089,2.329684,2.270366,...,0,0,0,0,0,0,0,0,0,0
27598,2.173532,2.394231,2.450000,2,0.0,4,5,2.234089,2.329684,2.270366,...,0,0,0,0,0,0,0,0,0,0
249419,2.333717,2.310843,2.157143,3,1.0,3,9,2.234089,2.329684,2.270366,...,0,0,0,0,0,0,0,0,0,0


In [47]:
X_train.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
143772,694725,2.342682,2.671518,2.287879,2,0.0,5,6,2.234089,2.329684,...,0,0,0,0,0,0,0,0,0,0
171425,285887,2.342682,2.419214,2.511111,3,0.0,4,8,2.234089,2.329684,...,0,0,0,0,0,0,0,0,0,0
153699,395621,1.927176,2.095652,1.875331,1,0.0,7,3,2.234089,1.811893,...,0,0,0,0,0,0,0,0,0,0
159185,1046162,2.164859,2.051546,1.823529,2,0.0,7,8,2.254037,2.329684,...,0,0,0,0,0,0,0,0,0,0
12708,865294,1.938499,2.240310,2.100017,2,0.0,13,6,2.234089,2.329684,...,0,0,0,0,0,0,0,0,0,0


# Remove Building ID

In [48]:
X_train = X_train.drop('building_id', axis=1)
X_val = X_val.drop('building_id', axis=1)

In [49]:
df.drop('building_id', axis=1, inplace=True)

In [50]:
corelation_df = df.corr()
high_corr_cols = corelation_df[corelation_df['damage_grade'] > 0.0].index.tolist()

In [51]:
high_corr_cols.remove('damage_grade')

In [52]:
X_train2 = X_train[high_corr_cols]
X_val2 = X_val[high_corr_cols]

# Model work

In [53]:
from scipy.stats import randint, uniform
params = {"criterion": ["gini", "entropy"],
          "splitter": ["best", "random"],
          "class_weight": ['balanced', None],
          "max_depth": [10,15,20,25],
          "min_samples_leaf": [7,10,15],
          "max_features": uniform(0.0, 1.0)} 

In [54]:
clf = DecisionTreeClassifier(random_state=0)

In [55]:
%%time
search = RandomizedSearchCV(clf, param_distributions=params, n_iter=200, scoring='f1_micro', cv=10, verbose=5, n_jobs=-1)

Wall time: 0 ns


In [80]:
%%time
search1 = search.fit(X_train, y_train)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   40.5s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  8.9min finished


Wall time: 8min 58s


In [56]:
%%time
search = search.fit(X_train2, y_train)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   45.3s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  3.3min finished


Wall time: 3min 17s


In [57]:
search.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 0.8829972551123816,
 'min_samples_leaf': 15,
 'splitter': 'best'}

In [58]:
clf = DecisionTreeClassifier(random_state=1,criterion= 'entropy'
                             , max_depth= 10, max_features= 0.9188411838536403
                             , min_samples_leaf= 15, splitter= 'best' )

In [59]:
%%time
scores = cross_val_score(clf, X_train2, y_train, cv=10, n_jobs=-1, scoring='f1_micro')
scores

Wall time: 1.5 s


array([0.74266117, 0.73772064, 0.73796048, 0.74323676, 0.74050269,
       0.73690522, 0.74002302, 0.73527437, 0.73748081, 0.73656946])

In [60]:
np.mean(scores)

0.7388334612432848

In [61]:
search.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 0.8829972551123816,
 'min_samples_leaf': 15,
 'splitter': 'best'}

In [62]:
#clf = DecisionTreeClassifier(random_state=1, max_depth=22, max_features=0.9847777938244793, min_samples_leaf=10, splitter='best')
clf = DecisionTreeClassifier(random_state=1, max_depth=9, max_features=0.8397147799867931, min_samples_leaf=2, splitter='best')

In [63]:
scores = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1, scoring='f1_micro')
scores

array([0.74210955, 0.7448916 , 0.74266117, 0.74112625, 0.74153396])

In [64]:
np.mean(scores)

0.742464504988488

In [66]:
rf = RandomForestClassifier(random_state=0, class_weight='balanced', n_estimators=200, criterion='gini',max_leaf_nodes=20)

In [67]:
rf_score = cross_val_score(rf, X_train, y_train, cv=10, scoring='f1_micro', n_jobs=-1)
rf_score

array([0.67334996, 0.66826554, 0.67426132, 0.67402149, 0.67263047,
       0.67334996, 0.67594014, 0.66975249, 0.67191097, 0.66826554])

In [68]:
np.mean(rf_score)

0.6721747889485801

In [69]:
%%time
rf_score2 = cross_val_score(rf, X_train, y_train, cv=10, scoring='f1_micro', n_jobs=-1)
rf_score2

Wall time: 1min 11s


array([0.67334996, 0.66826554, 0.67426132, 0.67402149, 0.67263047,
       0.67334996, 0.67594014, 0.66975249, 0.67191097, 0.66826554])

In [70]:
np.mean(rf_score2)

0.6721747889485801

In [92]:
X_val.shape

(52121, 38)

In [94]:
y_pred = search.predict(X_val)

In [95]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           1       0.64      0.45      0.53      4998
           2       0.73      0.83      0.78     29774
           3       0.74      0.63      0.68     17349

    accuracy                           0.73     52121
   macro avg       0.70      0.64      0.66     52121
weighted avg       0.73      0.73      0.72     52121



In [96]:
f1_score(y_val, y_pred, average='micro')

0.728727384355634

In [97]:
from scipy.stats import randint, uniform
estimators = [int(x) for x in np.linspace(50, 300, 30)]
max_feat = uniform(0.0, 1.0)
max_depth = [int(x) for x in np.linspace(5, 25, 10)]
min_samples_split = [2,5,10,15]
min_sample_leaf = [1,2,4,6,8]
bootstrap = [True, False]

In [98]:
rf_params = {'n_estimators': estimators,
               'max_features': max_feat,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_sample_leaf,
               'bootstrap': bootstrap}

In [99]:
rf2 = RandomForestClassifier(random_state=2)

In [100]:
%%time
search = RandomizedSearchCV(rf2, param_distributions=rf_params, n_iter=200, scoring='f1_micro', cv=10, verbose=2, n_jobs=-1)

Wall time: 0 ns


In [101]:
%%time
search = search1

Wall time: 0 ns


In [102]:
from sklearn.naive_bayes import MultinomialNB

In [104]:
from xgboost import XGBClassifier

In [105]:
%%time
xg = XGBClassifier(n_estimators=200, max_depth=8, gamma = 0.03874, scale_pos_weight=0, learning_rate =0.0459968, class_weight=class_weights)

Wall time: 0 ns


In [106]:
%%time
xg_scores = cross_val_score(xg, X_train, y_train, cv=5, n_jobs=5, scoring='f1_micro')
xg_scores

Wall time: 3min 39s


array([0.75306984, 0.75189467, 0.75342959, 0.75302187, 0.75235035])

In [108]:
np.mean(xg_scores)

0.7527532617037604

In [109]:
xg.fit(X_train, y_train);

E:\Tolu\PROGRAMS\Anaconda2\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:43:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "class_weight", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[00:43:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [110]:
xg_preds = xg.predict(X_val)

In [111]:
print(classification_report(xg_preds, y_val))

              precision    recall  f1-score   support

           1       0.52      0.67      0.58      3870
           2       0.84      0.75      0.79     33552
           3       0.64      0.75      0.69     14699

    accuracy                           0.74     52121
   macro avg       0.66      0.72      0.69     52121
weighted avg       0.76      0.74      0.75     52121



In [112]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, xg_preds)

array([[ 2580,  2337,    81],
       [ 1189, 24997,  3588],
       [  101,  6218, 11030]], dtype=int64)

In [113]:
clf_ada = AdaBoostClassifier(base_estimator=xg, n_estimators=100, random_state=0)

In [114]:
ada_scores = cross_val_score(clf_ada, X_train, y_train, cv=5, n_jobs=-1, scoring='f1_micro')
ada_scores

array([0.09653204, 0.09653204, 0.09653204, 0.09653204, 0.09655602])

# Test dataset

In [115]:
X_test = pd.read_csv('test_values.csv')

In [116]:
dummy_regions = pd.get_dummies(X_test['geo_level_1_id'], prefix='region1')

In [117]:
X_test = pd.concat((X_test, dummy_regions), axis=1)

In [118]:
X_test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,region1_21,region1_22,region1_23,region1_24,region1_25,region1_26,region1_27,region1_28,region1_29,region1_30
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,0,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,1,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,0,0,0,1,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


In [78]:
X_test.drop(['building_id', 'geo_level_1_id'], axis=1, inplace=True)

In [119]:
X_test['age'] = est.transform(X_test[['age']])

In [120]:
X_test[['geo_level_2_id','geo_level_3_id']] = X_test[['geo_level_2_id','geo_level_3_id']].astype('object')

In [121]:
test_cate_cols = X_test.select_dtypes(exclude='number').columns.tolist()
test_cate_cols

['geo_level_2_id',
 'geo_level_3_id',
 'land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

In [122]:
X_test[test_cate_cols] = te.transform(X_test[test_cate_cols])

ValueError: Unexpected input dimension 10, expected 11

In [ ]:
X_test.head()

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,...,region1_21,region1_22,region1_23,region1_24,region1_25,region1_26,region1_27,region1_28,region1_29,region1_30
0,2.676471,2.583328,3,0.0,7,6,2.234089,2.329684,2.270366,2.309677,...,0,0,0,0,0,0,0,0,0,0
1,2.185430,2.000590,2,0.0,13,5,2.234089,2.329684,2.270366,2.309677,...,0,0,0,0,0,0,0,0,0,0
2,2.193548,2.795228,2,0.0,4,5,2.234089,2.329684,2.270366,2.309677,...,0,1,0,0,0,0,0,0,0,0
3,1.528378,1.430769,1,0.0,19,3,2.234089,2.329684,1.574306,1.634633,...,0,0,0,0,0,1,0,0,0,0
4,2.670886,2.812500,3,0.0,8,7,2.234089,2.329684,2.319738,2.309677,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_train.shape, X_val.shape, X_test.shape

((208480, 68), (52121, 68), (86868, 68))

In [123]:
test_preds = xg.predict(X_test)

ValueError: Feature shape mismatch, expected: 38, got 70

In [ ]:
test_preds

array([3, 2, 3, ..., 2, 2, 2], dtype=int64)

In [ ]:
y_preds = pd.DataFrame(test_preds)
y_preds

,0
0,3
1,2
2,3
3,2
4,3
...,...
86863,2
86864,3
86865,2
86866,2


In [ ]:
X_test.head()

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,...,region1_21,region1_22,region1_23,region1_24,region1_25,region1_26,region1_27,region1_28,region1_29,region1_30
0,2.676471,2.583328,3,0.0,7,6,2.234089,2.329684,2.270366,2.309677,...,0,0,0,0,0,0,0,0,0,0
1,2.185430,2.000590,2,0.0,13,5,2.234089,2.329684,2.270366,2.309677,...,0,0,0,0,0,0,0,0,0,0
2,2.193548,2.795228,2,0.0,4,5,2.234089,2.329684,2.270366,2.309677,...,0,1,0,0,0,0,0,0,0,0
3,1.528378,1.430769,1,0.0,19,3,2.234089,2.329684,1.574306,1.634633,...,0,0,0,0,0,1,0,0,0,0
4,2.670886,2.812500,3,0.0,8,7,2.234089,2.329684,2.319738,2.309677,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_values = pd.read_csv('test_values.csv')
test_ids = test_values['building_id']
final_df = pd.concat([test_ids,y_preds], axis=1)

In [ ]:
final_df.head()

,building_id,0
0,300051,3
1,99355,2
2,890251,3
3,745817,2
4,421793,3


In [ ]:
final_df = final_df.rename(columns={0:"damage_grade"})

In [ ]:
final_df.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,3
3,745817,2
4,421793,3


In [114]:
my_submission = final_df
my_submission.to_csv('submission_DecisionTree.csv', index=False)